# Librerie

In [26]:
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt

# Funzioni 

In [52]:
mp_holistic = mp.solutions.holistic # Holistic model (to make our detection)

def mediapipe_detection(image, model):
    h = """prende in input un'immagine RGB e restituisce i risultati del rilevamento delle mani"""
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

mp_drawing = mp.solutions.drawing_utils # Drawing utilities (to draw them)
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

def extract_hand_landmark_coordinates(results):
    left_hand_landmarks = results.left_hand_landmarks
    right_hand_landmarks = results.right_hand_landmarks #Queste righe estraggono gli attributi left_hand_landmarks e right_hand_landmarks dall'oggetto results, che rappresentano i landmark delle mani sinistra e destra rilevate.
    left_hand_landmark_coordinates = []
    right_hand_landmark_coordinates = [] # Vengono inizializzate due liste vuote, left_hand_landmark_coordinates e right_hand_landmark_coordinates, che conterranno le coordinate dei landmark delle mani sinistra e destra.

    if left_hand_landmarks: # Questa condizione verifica se sono stati rilevati dei landmark per la mano sinistra.
        for landmark in left_hand_landmarks.landmark: #Questo ciclo itera su ciascun oggetto landmark nella lista dei landmark della mano sinistra.
            x = int(landmark.x * image.shape[1])
            y = int(landmark.y * image.shape[0])
            z = landmark.z
            left_hand_landmark_coordinates.append((x, y, z)) #Le coordinate (x, y, z) vengono aggiunte alla lista left_hand_landmark_coordinates.
    if right_hand_landmarks:
        for landmark in right_hand_landmarks.landmark:
            x = int(landmark.x * image.shape[1])
            y = int(landmark.y * image.shape[0])
            z = landmark.z
            right_hand_landmark_coordinates.append((x, y, z))

    return right_hand_landmark_coordinates, left_hand_landmark_coordinates

#La moltiplicazione per la larghezza e l'altezza dell'immagine serve a convertire le coordinate normalizzate 
# (valori compresi tra 0 e 1) fornite dai landmark in coordinate pixel all'interno dell'immagine.
#I landmark restituiti da MediaPipe sono normalizzati rispetto alle dimensioni dell'immagine di input, il che significa 
# che i valori x e y rappresentano le proporzioni rispetto alla larghezza e all'altezza dell'immagine. Moltiplicando questi 
# valori per le dimensioni effettive dell'immagine, otteniamo le coordinate in formato pixel.

def extract_real_z_values(hand_landmark_coordinates, depth_map):
    h = """prende in input le coordinate dei landmark della mano e la mappa di profondità (depth map) e 
    restituisce una lista di valori z reali corrispondenti a ciascun punto dei landmark."""

    real_z_values = []
    predicted_z_values = []

    for x, y, z in hand_landmark_coordinates:
        depth = depth_map[y, x]
        real_z = depth / 255.0 #Il valore di profondità (depth) viene normalizzato dividendo per 255.0, poiché i valori della mappa di profondità di solito sono rappresentati nell'intervallo [0, 255].
        real_z_values.append(real_z)
        predicted_z_values.append(z)
    return real_z_values, predicted_z_values

def calculate_accuracy(predicted_z_values, real_z_values, threshold): #calcola l'accuratezza delle predizioni dei valori z rispetto ai valori z reali.

    accurate_count = 0 #conta il numero di predizioni accurate.
    total_count = len(predicted_z_values) # Viene calcolato il numero totale di predizioni come la lunghezza della lista predicted_z_values.

    for predicted_z, real_z in zip(predicted_z_values, real_z_values):
        if abs(predicted_z - real_z) < threshold: #viene controllato se la differenza assoluta tra di essi è inferiore alla soglia threshold.
            accurate_count += 1 #Se sì, significa che la predizione è accurata.

    accuracy = accurate_count / total_count # viene calcolata l'accuratezza dividendo il conteggio delle predizioni accurate accurate_count per il numero totale di predizioni total_count.

    return accuracy

In [55]:
# Carica l'immagine RGB e la mappa di profondità
rgb_image = cv2.imread('Due.jpg')
depth_map = cv2.imread('001185 2.jpg', cv2.IMREAD_GRAYSCALE)

# Carica il modello di MediaPipe per il rilevamento delle mani
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Effettua la rilevazione delle mani con MediaPipe
    image, results = mediapipe_detection(rgb_image, holistic)

    #Per verificare visivamente la presenza di landmark prima dell'estrazione delle coordinate
    """# Draw landmarks
    draw_styled_landmarks(image, results)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))"""

    # Estrai le coordinate dei landmark della mano destra e sinistra
    right_hand_landmark_coordinates, left_hand_landmark_coordinates = extract_hand_landmark_coordinates(results)

    #Per verificare visivamente la presenza di landmark dopo dell'estrazione delle coordinate
    """for landmark in left_hand_landmark_coordinates:
    x, y, z = landmark
    cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # Disegna un cerchio verde intorno al landmark
    cv2.putText(image, f"({x}, {y}, {z})", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)  # Mostra le coordinate
    cv2.imshow('Landmarks', image)"""


    """# Estrai i valori di z reali e predetta dalla mappa di profondità per la mano destra
    real_z_values_right, predicted_z_values_right = extract_real_z_values(right_hand_landmark_coordinates, depth_map)

    # Estrai i valori di z reali e predetta dalla mappa di profondità per la mano sinistra
    real_z_values_left, predicted_z_values_left = extract_real_z_values(left_hand_landmark_coordinates, depth_map)


    # Calcola l'accuratezza per la mano destra
    threshold = 0.1  # Soglia per l'accuratezza
    accuracy_right = calculate_accuracy(predicted_z_values_right, real_z_values_right, threshold)
    print(f"Accuracy (Right Hand): {accuracy_right}")

    # Calcola l'accuratezza per la mano sinistra
    accuracy_left = calculate_accuracy(predicted_z_values_left, real_z_values_left, threshold)
    print(f"Accuracy (Left Hand): {accuracy_left}")

    # Visualizza i landmark delle mani sull'immagine
    for landmark in right_hand_landmark_coordinates:
        cv2.circle(image, landmark, 3, (0, 255, 0), -1)

    for landmark in left_hand_landmark_coordinates:
        cv2.circle(image, landmark, 3, (0, 0, 255), -1)

    # Mostra l'immagine a schermo
    cv2.imshow('OpenCV Feed', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()"""


In [45]:
image.shape

(1040, 1560, 3)